# Import des librairies et des fonctions de traitement des images

In [ ]:
from PIL import Image 
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import matplotlib.pyplot as plt

import os
import cv2
import random
import time
import datetime

In [ ]:
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (255, 255))  # Assurez-vous que la taille correspond à celle utilisée lors de l'entraînement
    img = img / 255.0  # Normalisez les valeurs des pixels
    img = np.expand_dims(img, axis=0)  # Ajoutez une dimension pour représenter le lot (batch)

    return img

In [ ]:
def load_data_multi_class(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):

            label = folder.lower()  # Utilisez le nom du dossier comme label
            print(label)
            for filename in os.listdir(folder_path):
                img_path = os.path.join(folder_path, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (255, 255))
                data.append(img)
                labels.append(label)

    data = np.array(data) / 255.0
    labels = np.array(labels)

    return data, labels

# On charge notre modèle

In [ ]:
from keras.models import load_model
#model.save('models/88-83.h5')

model = load_model('model.h5')
#model.summary()

# Evaluation des performances

## fonctions

In [ ]:
def predict_defect_multi_class(model, image_path):
    preprocessed_img = load_and_preprocess_image(image_path)
    predictions = model.predict(preprocessed_img)
    #print (model.predict(preprocessed_img))

    # Interprétez les prédictions
    class_labels = [ "0", "1"]
    predicted_class = class_labels[np.argmax(predictions)]

    return predicted_class

In [ ]:
def predict(dir):
    resultf=""
    nb_total=0
    nb_positif=0
    for filename in os.listdir(dir):
        #print(filename)
        nb_total+=1

        result = predict_defect_multi_class(model, dir +"/" + filename)
        if result.upper()==dir[4:].upper():
            nb_positif+=1

        resultf= resultf + "prédiction pour"+ filename + " : "+ str(result) +"\n"

    print (resultf)
    print(str((nb_positif*100)/nb_total) + " % de précision")


## résultats

On teste notre modèle sur 16 images sélectionnées de chaques catégories qui ne sont pas dans le dataset de train.

In [ ]:
predict("testok")

In [ ]:
predict("testbed_not_stick")

On remarque des performances bien plus basses pour le défaut "bed not stick", cependant celui ci est difficile à décerner visuellement et entraine rapidement la création de spaguettis qui sont eux bien détectés par notre modèle.

In [ ]:
predict("testspaghetti")

# Test avec de nouvelles images aléatoires

In [ ]:
def vignette_random(dir):
  L=[]
  for filename in os.listdir(dir):
    L.append(filename)
  file_alea = L[random.randint(0, (len(L)-1))]
  image_path = dir +"/" + file_alea
  result = predict_defect_multi_class(model, image_path)
  img = Image.open(image_path)
  plt.imshow(img)
  plt.axis('off')  # Masquer les axes
  plt.show()

  print (result)

In [ ]:
vignette_random("testok")

In [ ]:
vignette_random("testspaghetti")

In [ ]:
vignette_random("testbed_not_stick")

# Prédiction à partir de captures vidéos en direct

On prend une photo avec une webcam USB et on process la capture pour faire de la détéction de défaut avec notre modèle

## Fonctions

In [ ]:
def capture():
  # Ouvrir la webcam (la webcam par défaut a l'ID 0)
  cap = cv2.VideoCapture(0)

  # Vérifier si la webcam est ouverte correctement
  if not cap.isOpened():
      print("Erreur: Impossible d'ouvrir la webcam.")
      exit()

  # Capturer une image
  time.sleep(0.5)
  ret, frame = cap.read()

  # Sauvegarder l'image capturée
  if ret:
      cv2.imwrite("photo_capturee.jpg", frame)
      print("Photo capturée avec succès.")

      result = predict_defect_multi_class(model, 'photo_capturee.jpg')
      img = Image.open('photo_capturee.jpg')
      plt.imshow(img)
      plt.axis('off')  # Masquer les axes
      now = datetime.datetime.now().strftime("%H:%M:%S")
      

      plt.show()
      print(f"Prédiction pour la capture caméra : {result}")
      print(f"à {now}")
  
  
  else:
      print("Erreur lors de la capture de la photo.")

  # Libérer la webcam
  cap.release()



In [ ]:
def capture_boucle():
    try:
        while True:
            # Exécute la fonction capture()
            capture()
            print()

            
            # Pause de x secondes
            time.sleep(0.5)

    except KeyboardInterrupt:
        print("\nInterruption de l'utilisateur. Fin du script.")

## Démo

In [ ]:
capture()

In [ ]:
capture_boucle()